In [2]:
import numpy as np
# import scipy.linalg as LA 
# import copy
# %matplotlib widget
# import matplotlib.pyplot as plt

from ncon import ncon
from timeit import default_timer as timer



# Tensor operations


### E1: Defining generic tensors  
Here we will start studying generic tensor networks and their contraction.

Part of any tensor network study is to understand how to contract some of the parts of the tensor networks, and eventually simplify them. 

The first task is to initialize tensors, we did this already for vectors and tensors are no different. 

Let' s create 

- a random complex tensor with four legs (an order 4 tensor) with size 2, 3, 4, 5 and call it A
- an order 3 tensor with size 4 5 6 and call it B
- a 5x5 identity matrix

In [3]:
A = np.random.rand(2,3,4,5) +1j*np.random.rand(2,3,4,5)
B = np.random.rand(4,5,6) +1j*np.random.rand(4,5,6)
C = np.eye(5)  # should be same as np.eye(5,5)


### E2: Some special tensors

Other special tensors are those made by all ones, that can be multiplied by an arbitrary constant, thus obtaining a tensor where each element is the same.

-Create one that has order 4 and dimension 2, 4, 2, 4, and call it D

Finally when a tensor only has few non-zero elements, one can create a tensor made of all zeros and fill the desider elements.

-Create a tensor with order 2 made of zero and fill the first element with a random complex number

In [4]:
# tensor of 1's, order 4, dims: 2-by-4-by-2-by-4
D = np.ones((2,4,2,4), dtype=complex)
D *= (np.random.rand()+1j*np.random.rand())

# matrix of 0's, order 2, dims: 3-by-5
F = np.zeros((3,5), dtype=complex)
F[0,0] = np.random.rand()+1j*np.random.rand()

### E3: Permuting and reshaping 

Some common operations on tensors are 

-Reordering the legs, ie. permuting them, which incurs in a computational cost proportional to the size of the tensor

-Grouping or splitting the legs, which does not have a relevant computational cost (for large tensors) since it only changes the labels used to address the elements 

For example implement the above permutation and reshaping  for the tensor A  and B defined in the figure 

<img src="reshape_permute.png" width=400>


In [5]:
At = A.transpose(3,0,1,2)
Bt = B.reshape(4,30)

print(f"{np.shape(A)=}")
print(f"{np.shape(At)=}")
print(f"{np.shape(Bt)=}")

np.shape(A)=(2, 3, 4, 5)
np.shape(At)=(5, 2, 3, 4)
np.shape(Bt)=(4, 30)


## Tensor contractions

We now enter the realm of tensor contractions. 

First of all, the basic rule: contracting two tensors means summing the product of the corresponding tensor elements, it is a generalization of matrix multiplication

$ M_{ik}  =\sum_j A_{ij} * B_{jk}$

We will start by contracting two tensors. This can be done in several ways.

Define a new tensor $G$ with order 4 and dimensions 3,6,7,5

Now contract it with the $A$ tensor with dimensions (2,3,4,5) defined above on the second and fourth leg, ie. 

$ K_{ijkl} = \sum_{mn} A_{imjn} G_{mkln} $

First compute it using for loops. Note that you need to loop over all the involved legs, hence the computational cost! 

In [6]:

G = np.random.rand(3,6,7,5) +1j*np.random.rand(3,6,7,5) 

K = np.zeros((A.shape[0],A.shape[2],G.shape[1],G.shape[2]), dtype=complex)

print(np.shape(A), np.shape(G))

for i in range(A.shape[0]):
    for m in range(A.shape[1]):
        for j in range(A.shape[2]):
            for n in range(A.shape[3]):
                for k in range(G.shape[1]):
                    for ll in range(G.shape[2]):
                        K[i,j,k,ll] += A[i,m,j,n]*G[m,k,ll,n]
                       
                    


(2, 3, 4, 5) (3, 6, 7, 5)


### E4: Tensor products as matrix multiplications 

Now repeat the same operation by transforming the two tensors into matrix and then performing a matrix multiplication, 
you will need to transpose and reshape accordingly.

Compare that the two methods provide the same result.



In [7]:
Ap = A.transpose(0,2,1,3)
Gp = G.transpose(0,3,1,2)
App = Ap.reshape(A.shape[0]*A.shape[2],A.shape[1]*A.shape[3])
Gpp = Gp.reshape(G.shape[0]*G.shape[3],G.shape[1]*G.shape[2])
Kpp = App @ Gpp            
K_tilde= Kpp.reshape(A.shape[0],A.shape[2],G.shape[1],G.shape[2])

np.allclose(K,K_tilde)


True

### Contractions using einsum 
There are more tools to make our lives easier, such as np.einsum, which (as the name suggests)
is inspired by the Einstein notation. We can make the same contraction above as 

In [8]:
K_einsum = np.einsum("imjn,mkln->ijkl", A, G)

np.allclose(K,K_einsum)

True

### Contractions using ncon 

Finally, we can also use ncon (network contractor), which can also work with networks of more tensors.

From the manual:

Network contractor ‘ncon’:

The ‘ncon’ function is a useful tool to lessen the programming effort required to implement a tensor network contraction. This function works by automatically performing a desired sequence of permutes, reshapes and matrix multiplications required to evaluate a tensor network. The ‘ncon’ code and detailed instructions for its usage can be found here, or alternatively the code is also presented on the example code page. The first step in using ‘ncon’ to evaluate a network is to make a labelled diagram of the network such that:​

​

Each internal index is labelled with a unique positive integer (typically sequential integers starting from 1, although this is not necessary).

​

External indices of the diagram (if there are any) are labelled with sequential negative integers [-1,-2,-3,…] which denote the desired index order on the final tensor (with -1 as the first index, -2 as the second etc).

​

Following this, the ‘ncon’ routine is called as follows,

​

 

OutputTensor = ncon(TensorArray, IndexArray),

​

 

with input arguments defined:

TensorArray: 1D cell array containing the tensors comprising the network

​

IndexArray: 1D cell array of vectors, where the kth element is a vector of the integer labels from the diagram on the kth tensor from ‘TensorArray’ (ordered following the corresponding index order on this tensor).

In [9]:
K_ncon = ncon([A,G], [[-1,1,-2,2],[1,-3,-4,2]])

np.allclose(K_ncon, K)

True

### Computational cost of tensor contractions. 

As you have seen, contracting tensors comes at a cost.
When your network to contract include more than two tensors it is computationally advantageous to break the contraction into pairwise contractions. For example if you need to contract three tensors, and you do it in a single shot (by using for loops) you incur into a higher computational cost. Try it with the simple matrix product A*B*C, with all three being d*d matrices and, eg. d=70


In [10]:
d = 60
A = np.random.rand(d,d) 
B = np.random.rand(d,d)
C = np.random.rand(d,d)

In [11]:
# Evaluate network via summation over internal indices
F0 = np.zeros((d,d))

for di in range(d):
    for dj in range(d):
        for dk in range(d):
            for dl in range(d):
                F0[di,dj] += A[di,dk]*B[dk,dl]*C[dl,dj]
            

In [12]:

# Evaluate network via sequence of binary contractions
F1 = (A @ B) @ C

In [13]:
np.allclose(F0,F1)

True

### Some timing comparison

In [14]:
d = 10
A = np.random.rand(d,d,d) 
B = np.random.rand(d,d,d)
C = np.random.rand(d,d,d)

##### Evaluate network via index summation
def tempfunct(A,B,C,d):
    D0 = np.zeros((d,d,d))
    for b1 in range(d):
        for a2 in range(d):
            for c3 in range(d):
                for a1 in range(d):
                    for a3 in range(d):
                        for c1 in range(d):
                            D0[b1,a2,c3] = D0[b1,a2,c3]+A[a1,a2,a3]*B[b1,a1,c1]*C[c1,a3,c3]
    return D0

t0 = timer()
D0 = tempfunct(A,B,C,d)
t_sum = timer() - t0

##### Evaluate network using reshape and permute
def tempfunct2(A,B,C,d):
    Xmid = (B.transpose(0,2,1).reshape(d**2,d) @ A.reshape(d,d**2)).reshape(d,d,d,d)
    D1 = (Xmid.transpose(0,2,1,3).reshape(d**2,d**2) @ C.reshape(d**2,d)).reshape(d,d,d)
    return D1

t0 = timer()
D1 = tempfunct2(A,B,C,d)
t_res = timer() - t0

##### Evaluate using ncon
t0 = timer()
D2 = ncon([A,B,C],[[1,-2,2],[-1,1,3],[3,2,-3]]) #, cont_order = [1,2,3])
t_ncon = timer() - t0

##### Compare
tdiffs = [max(abs(D0-D1).flatten()),max(abs(D1-D2).flatten()),max(abs(D2-D0).flatten())]
ttimes = [t_sum, t_res, t_ncon]
print(ttimes)

[1.2159808999986126, 0.00025629999981902074, 0.00036040000122739]


### SVD and factorizations

We can use SVDs (or other factorizations such as QR) to separate a single tensor into multiple ones.

To get an idea, work with matrices. 
Build a random (40x28) matrix and SVD it. What is the maximum rank of the matrix ? 

In [15]:
M = np.random.rand(40,28) +1j*np.random.rand(40,28)
u,s,vd = np.linalg.svd(M, full_matrices=False)
print(np.shape(u),np.shape(s),np.shape(vd))

(40, 28) (28,) (28, 28)


In [16]:
q,r = np.linalg.qr(M)
print(np.shape(q),np.shape(r))
np.allclose(q @ r ,M)

(40, 28) (28, 28)


True

### Reconstruction 

Try to reconstruct the original tensor, see that nothing got lost. Use einsum or ncon

Hint: recall that - while it is stored as a vector, S should be actually thougt of as a diagonal matrix

In [17]:
sm = np.diag(s)
print(np.shape(sm))

M_ncon = ncon([u,sm,vd],[[-1,1],[1,2],[2,-2]])
M_einsum = np.einsum("ij,jk,kl->il",u,sm,vd)
print(np.allclose(M_ncon,M))
print(np.allclose(M_einsum,M))

(28, 28)
True
True


### Truncation 

We can also use SVDs to truncate though. For this, we can discard the smallest singular values. 

What will the error be if we try to reconstruct the tensor ? 

In [18]:
cut = 20 

sm = np.diag(s[:cut])

print(f"{np.linalg.norm(s[cut:])=}")

print(np.shape(sm))

M_ncon = ncon([u[:,:cut],sm,vd[:cut,:]],[[-1,1],[1,2],[2,-2]])
print(f"{np.allclose(M_ncon,M)=}")

print(f"{np.linalg.norm(M_ncon - M)=}")

np.linalg.norm(s[cut:])=np.float64(3.036062584529339)
(20, 20)
np.allclose(M_ncon,M)=False
np.linalg.norm(M_ncon - M)=np.float64(3.036062584529339)
